In [1]:
import pandas as pd
from pathlib import Path

In [2]:
model = 'gemma2b'
dist_folder = 'no-distil-test'
filename = 'FINAL_Completed(0 : 1319).csv'

In [5]:
basepath = (Path() / '..' /'..'/ 'logs' / model / dist_folder).resolve()

In [6]:
df = pd.read_csv(basepath / filename)

In [7]:
for i, row in df.iterrows():
    df.loc[i, 'model_answer'] = row['model_answer'].removeprefix(row['question'])
    df.loc[i, 'model_answer'] = df.loc[i, 'model_answer'].removeprefix('\n Please respond with just the answer. The answer is: ')
    df.loc[i, 'extracted_answer'] = row['answer'].split('####')[1].strip()

Ideas:
- Use equals sign and split by the last element
- Empty answers (4, 26, 28)

In [8]:
ans = 0
tfor = 0
eqs = 0
noex = 0
noans = 0
ids = []
for i, a in df['model_answer'].items():
    if not a:
        noans += 1
    elif '\n\n' in a.lower():
        ans += 1
    else:
        noex += 1
        ids.append(i)

print(f'Empty string: {noans}')
print(f'Answer is: {ans}')
# print(f'Therefore: {tfor}')
# print(f'Equals: {eqs}')
print(f'No extraction: {noex}')
ids[:10]

Empty string: 0
Answer is: 1200
No extraction: 119


[6, 21, 28, 40, 51, 84, 93, 96, 150, 157]

In [11]:
def save_df(df : pd.DataFrame, model : str, dist_folder : str, edition : str):
    base_path = Path()
    base_path = base_path / '..' /'..'/ 'logs' / model / dist_folder / f'{edition}.csv'
    df.to_csv(base_path)

In [12]:
df['model_ex1'] = ''
start = 0
c = 0
for i, row in df[start:].iterrows():
    try:
        df.loc[i, 'model_ex1'] = row['model_answer'].split('\n\n')[0]
    except:
        df.loc[i, 'model_ex1'] = row['model_answer']
save_df(df, 'gemma2b', 'no-distil-test', 'extract_1')
    

In [13]:
df['model_ex1_complete'] = df['model_ex1']
df['model_ex1_complete'] = df['model_ex1_complete'].map(lambda x : x.isnumeric())

In [14]:
df_false = df[df['model_ex1_complete'] == False]

In [15]:
dol = 0
perc = 0
spac = 0
no = 0

id_nos = []
for i, val in df_false['model_ex1'].items():
    if '$' in val:
        dol += 1
    elif '%' in val:
        perc += 1
    elif ' ' in val:
        spac += 1
    else:
        no += 1
        id_nos.append(i)
    

print(f'$$$ : {dol}')
print(f'% : {perc}')
print(f'Space : {spac}')
print(f'N/A : {no}')
print(id_nos[:10])

$$$ : 365
% : 19
Space : 196
N/A : 31
[23, 43, 66, 88, 133, 136, 165, 193, 328, 337]


In [16]:
df['model_ex2'] = df['model_ex1']
for i, row in df[df['model_ex1_complete'] == False].iterrows():
    if '$' in row['model_ex1']:
        df.loc[i, 'model_ex2'] = row['model_ex1'].removeprefix('$')
        if df['model_ex2'][i].isnumeric():
            continue
    if ' ' in row['model_ex1']:
        vls = row['model_ex1'].split(' ')
        v = ''
        for vs in vls:
            try:
                v = float(vs)      
                df.loc[i, 'model_ex2'] = v 
            except:
                continue
        if v == '':
            df.loc[i, 'model_ex2'] = row['model_ex1'] 

In [17]:
for i, row in df.iterrows():
    try:
        df.loc[i, 'model_ex2'] = float(row['model_ex2'])

    except:
        pass
    try:
        df.loc[i, 'extracted_answer'] = float(row['extracted_answer'])
    except:
        pass
    df.loc[i, 'model_ex2_t'] = type(df['model_ex2'][i])
    df.loc[i, 'extracted_answer_t'] = type(df['extracted_answer'][i])

In [18]:
df['extracted_answer_t'].value_counts()

extracted_answer_t
<class 'float'>    1305
<class 'str'>        14
Name: count, dtype: int64

In [19]:

for i, row in df[df['model_ex2_t'] != float].iterrows():
    inp = input(f'{i} : What is the extracted answer : {row["model_ex2"]}')
    try:
        df.loc[i, 'model_ex2'] = float(inp)
    except:
        print(f'{i} could not be parsed')


In [20]:

for i, row in df[df['extracted_answer_t'] != float].iterrows():
    inp = input(f'{i} : What is the extracted answer : {row["extracted_answer"]}')
    try:
        df.loc[i, 'extracted_answer'] = float(inp)
    except:
        print(f'{i} could not be parsed')


In [21]:
for i, row in df.iterrows():
    df.loc[i, 'model_ex2_t'] = type(df['model_ex2'][i])
    df.loc[i, 'extracted_answer_t'] = type(df['extracted_answer'][i])
    

In [24]:
df['correct'] = df['model_ex2'] == df['extracted_answer']

In [25]:
df['correct'].value_counts()

correct
False    1256
True       63
Name: count, dtype: int64

In [26]:
save_df(df, 'gemma2b', 'no-distil-test', 'FINAL_RESULTS')